In [1]:
from app import Errors
import itertools
import pandas as pd
import os
from math import pi
from datetime import datetime, timedelta

In [2]:
from bokeh.models import ColumnDataSource
from bokeh.plotting import show, output_notebook, figure
from bokeh.palettes import Category20c
from bokeh import palettes
from bokeh import layouts
from bokeh.transform import cumsum
output_notebook()

Loading BokehJS ...

# Compute time intervals where the ptl was online

In [3]:
from app import Connectivity
from datetime import datetime, timedelta
from plots import plot_connection_times

In [4]:
since = datetime.now() - timedelta(days=3)
until = datetime.now()
data = Connectivity.connection_times(since, until)

In [5]:
device = 'PTL_RD_ES_012'
device_data = data.loc[device]
data_source = ColumnDataSource(device_data)
x_range = device_data.begin.min(), device_data.end.max()
fig = figure(x_axis_type="datetime", x_range=x_range, y_range=(0, 0.25),
             plot_height=50, plot_width=800)
fig.hbar(y=0.125, left='begin', right='end', height=0.25, color='color', source=data_source)
fig.yaxis.visible=False
fig.toolbar.logo = None
fig.toolbar_location = None
show(fig)

In [7]:
device = 'PTL_RD_ES_012'
device_data = data.loc[device]
fig = plot_connection_times(device_data)
show(fig)

# Presense plot

In [ ]:
# TODO:
# test time range: begin=2020-03-14 00:00:00&end=2020-03-16 09:27:58 for PTL_RD_AT_001
# we get a overlapping gray bar in this case (and many other, but not always)
# there is a similar bug for the connection times

In [3]:
from app import PresenceDetectorStatistics

In [28]:
since = datetime.now() - timedelta(days=5)
until = datetime.now() - timedelta(days=4)
data = PresenceDetectorStatistics().on_off_timeseries(since, until)

In [62]:
colors = ['#eeeeee', '#ffcc00']

In [63]:
data['color'] = data.value.apply(lambda x: colors[x])

In [65]:
device = 'PTL_RD_AT_001'
device_data = data.loc[device].dropna()
data_source = ColumnDataSource(device_data)
data_source_off = ColumnDataSource(device_data[device_data.value == 0])
data_source_on = ColumnDataSource(device_data[device_data.value == 1])
x_range = device_data.begin.min(), device_data.end.max()
fig = figure(x_axis_type="datetime", x_range=x_range,
             plot_height=50, plot_width=800)
fig.hbar(y=0.125, left='begin', right='end', height=0.25, color='color', source=data_source)
fig.hbar(y=0.125, left='begin', right='end', height=0.5, color='color', source=data_source_on)
fig.yaxis.visible=False
fig.toolbar.logo = None
fig.toolbar_location = None
fig.xgrid.grid_line_color = None
fig.ygrid.grid_line_color = None
fig.outline_line_color = None
show(fig)

In [47]:
device_data[device_data.value == 0]

,begin,value,keep_row,duration,end,color
8,2020-03-11 09:38:16.443710,0,-1.0,08:10:18.800111,2020-03-11 17:48:35.243821,#ff0000
10,2020-03-11 17:49:06.687284,0,-1.0,08:05:45.479978,2020-03-12 01:54:52.167262,#ff0000
12,2020-03-12 01:55:22.830804,0,-1.0,00:07:58.814838,2020-03-12 02:03:21.645642,#ff0000


In [48]:
device_data[device_data.value == 1]

,begin,value,keep_row,duration,end,color
7,2020-03-11 07:31:02.291241,1,1.0,02:07:14.152469,2020-03-11 09:38:16.443710,#00ff00
9,2020-03-11 17:48:35.243821,1,1.0,00:00:31.443463,2020-03-11 17:49:06.687284,#00ff00
11,2020-03-12 01:54:52.167262,1,1.0,00:00:30.663542,2020-03-12 01:55:22.830804,#00ff00
13,2020-03-12 02:03:21.645642,1,1.0,00:00:18.908597,2020-03-12 02:03:40.554239,#00ff00


# Debug

In [3]:
from app import PresenceDetectorStatistics

In [6]:
since = datetime.now() - timedelta(days=1)
until = datetime.now()
data = PresenceDetectorStatistics().on_off_timeseries(since, until)
debug_data = PresenceDetectorStatistics().debug_on_off_timeseries(since, until)

In [7]:
debug_data.loc['PTL_RD_AT_000']

,timestamp,value
9,2020-03-17 09:55:42.802864,0
10,2020-03-17 10:04:04.718547,1
11,2020-03-17 10:04:10.930918,0
12,2020-03-17 11:12:24.241110,0
13,2020-03-17 11:26:05.284190,1
14,2020-03-17 11:26:25.046892,0


In [8]:
data.loc['PTL_RD_AT_000']

,begin,value,keep_row,duration,end
0,2020-03-17 09:55:42.802864,0,-1.0,00:08:21.915683,2020-03-17 10:04:04.718547
1,2020-03-17 10:04:04.718547,1,1.0,01:08:19.522563,2020-03-17 11:12:24.241110
3,2020-03-17 11:12:24.241110,0,-1.0,00:13:41.043080,2020-03-17 11:26:05.284190
4,2020-03-17 11:26:05.284190,1,1.0,00:00:19.762702,2020-03-17 11:26:25.046892
5,2020-03-17 11:26:25.046892,0,NaN,NaT,NaT


In [16]:
d = debug_data
d['keep_row'] = d.groupby('device').value.diff(periods=1)
d = d[d.keep_row != 0.0]
d.loc['PTL_RD_AT_000']

,timestamp,value,keep_row
9,2020-03-17 09:55:42.802864,0,NaN
10,2020-03-17 10:04:04.718547,1,1.0
11,2020-03-17 10:04:10.930918,0,-1.0
13,2020-03-17 11:26:05.284190,1,1.0
14,2020-03-17 11:26:25.046892,0,-1.0


In [4]:
since = datetime.now() - timedelta(days=1)
until = datetime.now()
data = PresenceDetectorStatistics().on_off_timeseries(since, until, device="PTL_RD_ES_009")

AttributeError: 'DataFrame' object has no attribute 'value'

In [8]:
from app import db
from db import InstructionPackage
device="PTL_RD_ES_009"
ip = InstructionPackage
query = db.session.query(ip.device, ip.timestamp, ip.value) \
    .filter(ip.source.contains("Lullaby")) \
    .filter(ip.instruction == "MODE") \
    .filter(ip.target == "POWER") \
    .filter(ip.timestamp >= since) \
    .filter(ip.timestamp <= until) \
    .order_by(ip.device, ip.timestamp)

if device:
    query = query.filter(ip.device == device)

data = pd.DataFrame(query.all())

In [9]:
data

""


In [10]:
data.empty

True